In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
#style.use('fivethirtyeight')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/nba-shot-logs/shot_logs.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Convert all column names to lowercase for ease of typing
df.columns = df.columns.str.lower()

In [ ]:
df['shot_result']

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(axis=1, inplace =True)

In [ ]:
df['matchup']
df['game_clock']
df['touch_time'].min()

####Dropping Outliers

##### A touch time of 163.6 is not possible. There're probably outliers in the touch_time column

In [ ]:
sns.boxplot(df['touch_time'])

In [ ]:
Q1 = df['touch_time'].quantile(0.25)
Q3 = df['touch_time'].quantile(0.75)

IQR = Q3 - Q1

print(Q1)
print(Q3)
print(IQR)

In [ ]:
Lower_Whisker = Q1 -  1.5*IQR
Upper_Whisker = Q3 + 1.5*IQR
print(Lower_Whisker, Upper_Whisker)

In [ ]:
outliers = df[(df.touch_time < Lower_Whisker) | (df.touch_time > Upper_Whisker)]

In [ ]:
df = df.drop(outliers.index)

In [ ]:
sns.boxplot(df['touch_time'])

#### There are still outliers in the dataset. Since it's impossible to have a negative touch time, we remove touch times less than 0

In [ ]:
outliers = df[(df.touch_time < 0)]

In [ ]:
df = df.drop(outliers.index)

In [ ]:
sns.boxplot(df['touch_time'])

In [ ]:
df['location'].value_counts()

In [ ]:
#df['home_game'] = np.where(df['location'] == 'H', 1, 0)
df['location'] = np.where(df['location'] == 'H', 1, 0)

In [ ]:
#df['game_won'] = np.where(df['w'] == 'W', 1, 0)
df['w'] = np.where(df['w'] == 'W', 1, 0)

In [ ]:
#df['w'] = np.where(df['w'] == 'W', 1, 0)

In [ ]:
df.shot_result = df.shot_result.map({"made":1,"missed":0})

In [ ]:
# converting the game_clock values to the seconds
df.game_clock = df.game_clock.apply(lambda x: int(x.split(":")[0])*60 + int(x.split(":")[1]))

In [ ]:
df.info()

In [ ]:
df['pts'].value_counts()

In [ ]:
df['matchup']

In [ ]:
# Extract game date and convert to datetime object
df['date'] = df['matchup'].str[:12]
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y')

In [ ]:
df = df.set_index('date')

In [ ]:
#df = df.drop(columns = ['game_id', 'matchup', 'closest_defender', 'closest_defender_player_id', 'player_name', 'player_id'])

In [ ]:
df = df.sort_index()

In [ ]:
df

In [ ]:
#all_players = list(df['player_name'].unique())

In [ ]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(20, 20))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot = True, fmt=".1f", mask=mask, cmap='twilight', vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Drop irrelevant and highly correlated Columns

In [ ]:
df = df.drop(columns = ['player_id', 'player_name','matchup', 'game_id', 'closest_defender_player_id', 'closest_defender', 'fgm', 'pts'], axis=1)

In [ ]:
df.info()

In [ ]:
df['shot_result'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X  = df.drop('shot_result', axis=1)
y = df['shot_result']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Logistic Regression

In [ ]:
logistic = LogisticRegression()

logistic.fit(X_train, y_train)

c = logistic.predict(X_test)

# Test score
score_logistic = logistic.score(X_test, y_test)
print('test accuracy:')
print(score_logistic)
print('train accuracy:')
print(logistic.score(X_train, y_train))

print(classification_report(y_test,prediction_lr))
metrics.plot_roc_curve(logistic, X_test, y_test)

confusion_matrix(y_test, prediction_lr)
pd.crosstab(y_test, prediction_lr, rownames = ['Actual'], colnames =['Predicted'], margins = True)

Random Forest

In [ ]:
gb = GradientBoostingClassifier(max_features='auto',
                       n_estimators=10, random_state=42, max_depth=5, min_samples_leaf=100, learning_rate = 0.08) 

gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

# Test score
score_gb = gb.score(X_test, y_test)
print('test accuracy:')
print(score_gb)
print('train accuracy:')
print(gb.score(X_train, y_train))

#Classification Report
report = classification_report(y_test, y_pred, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
print(df_classification_report)
metrics.plot_roc_curve(gb, X_test, y_test)

confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)

Random Forest

In [ ]:
rf = RandomForestClassifier(max_features='auto',
                       n_estimators=10, random_state=42, max_depth=5, min_samples_leaf=100) 

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Test score
score_rf = rf.score(X_test, y_test)
print('test accuracy:')
print(score_rf)
print('train accuracy:')
print(rf.score(X_train, y_train))

#Classification Report
report = classification_report(y_test, y_pred, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
print(df_classification_report)
metrics.plot_roc_curve(rf, X_test, y_test)

confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)

Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier(max_features='auto',
                       n_estimators=10, random_state=42, max_depth=5, min_samples_leaf=100, learning_rate = 0.08) 

gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

# Test score
score_gb = gb.score(X_test, y_test)
print('test accuracy:')
print(score_gb)
print('train accuracy:')
print(gb.score(X_train, y_train))

#Classification Report
report = classification_report(y_test, y_pred, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
print(df_classification_report)
metrics.plot_roc_curve(gb, X_test, y_test)

confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)

In [ ]:
gb = GradientBoostingClassifier(max_features='auto',
                       n_estimators=8, random_state=42, max_depth=5, min_samples_leaf=10, learning_rate = 0.1) 

gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)

# Test score
score_gb = gb.score(X_test, y_test)
print('test accuracy:')
print(score_gb)
print('train accuracy:')
print(gb.score(X_train, y_train))

#Classification Report
report = classification_report(y_test, y_pred, output_dict=True)
df_classification_report = pd.DataFrame(report).transpose()
print(df_classification_report)
metrics.plot_roc_curve(gb, X_test, y_test)

confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)

In [ ]:
pip install alibi

In [ ]:
target_names = (['made', 'missed'])

In [ ]:
target_names = np.array(target_names)

In [ ]:
import alibi

In [ ]:
from alibi.explainers import ALE, plot_ale
from alibi.explainers.ale import ALE, plot_ale

#### ALE on Logistic Regression in logit space

In [ ]:
logit_fun_lr = logistic.decision_function

In [ ]:
#logit_ale_lr = ALE(logit_fun_lr, feature_names=X.columns, target_names=target_names)

In [ ]:
logit_ale_lr = ALE(logit_fun_lr, feature_names=X.columns)

In [ ]:
logit_exp_lr = logit_ale_lr.explain(X_train.values)

In [ ]:
plot_ale(logit_exp_lr, fig_kw={'figwidth': 15, 'figheight': 10}, sharey=None);

In [ ]:
plot_ale(logit_exp_lr, features = ['shot_dist'], fig_kw={'figwidth': 15, 'figheight': 10}, sharey=None);

In [ ]:
#plot_ale(logit_exp_lr)

#### ALE on Logistic Regression in probability space

In [ ]:
proba_fun_lr = logistic.predict_proba

In [ ]:
#proba_ale_lr = ALE(proba_fun_lr, feature_names=df.columns, target_names=target_names)
proba_ale_lr = ALE(proba_fun_lr, feature_names=X.columns)

In [ ]:
proba_exp_lr = proba_ale_lr.explain(X_train.values)

In [ ]:
plot_ale(proba_exp_lr, fig_kw={'figwidth': 10, 'figheight': 10});

ALE for Random Forest

In [ ]:
#logit_fun_rf = rf.decision_function
proba_fun_rf = rf.predict_proba

In [ ]:
#logit_ale_rf = ALE(logit_fun_rf, feature_names=X.columns)
proba_ale_rf = ALE(proba_fun_rf, feature_names=X.columns)

In [ ]:
#logit_exp_gb = logit_ale_gb.explain(X_train.values)
proba_exp_rf= proba_ale_rf.explain(X_train.values)

ALE on Random Forest in logit space

In [ ]:
#plot_ale(logit_exp_gb, n_cols=2, fig_kw={'figwidth': 10, 'figheight': 10});

ALE on Random Forest in the probability space

In [ ]:
plot_ale(proba_exp_rf, n_cols=2, fig_kw={'figwidth': 10, 'figheight': 10});

#### ALE for gradient boosting

In [ ]:
logit_fun_gb = gb.decision_function
proba_fun_gb = gb.predict_proba

In [ ]:
logit_ale_gb = ALE(logit_fun_gb, feature_names=X.columns)
proba_ale_gb = ALE(proba_fun_gb, feature_names=X.columns)

In [ ]:
logit_exp_gb = logit_ale_gb.explain(X_train.values)
proba_exp_gb = proba_ale_gb.explain(X_train.values)

#### ALE for Gradient Boosting in logit space

In [ ]:
plot_ale(logit_exp_gb, n_cols=2, fig_kw={'figwidth': 10, 'figheight': 10});

#### ALE for Gradient Boosting in probability space

In [ ]:
plot_ale(proba_exp_gb, n_cols=2, fig_kw={'figwidth': 10, 'figheight': 10});

In [ ]:
gb.feature_importances_

In [ ]:
pd.DataFrame(gb.feature_importances_, index = X.columns)

In [ ]:
def plot_feature_importance(importance,names,model_type):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    names = X.columns
    feature_names = np.array(names)
    #feature_names = X_train.columns
    
    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    
    #Define size of bar plot
    plt.figure(figsize=(10,8))
    
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'], palette = 'Blues')

    #Add chart labels
    plt.title(model_type + ' FEATURE IMPORTANCE')

    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
plot_feature_importance(gb.feature_importances_,X.columns,'Random Forest Classifier')

### Comparing ALE between models

In [ ]:
fig, ax = plt.subplots(1,1 , figsize=(8, 4), sharey='row');
plot_ale(proba_exp_lr, features=['shot_dist'], ax=ax, line_kw={'label': 'Linear regression'});
plot_ale(proba_exp_rf, features=['shot_dist'], ax=ax, line_kw={'label': 'Random forest'});
plot_ale(proba_exp_gb, features=['shot_dist'], ax=ax, line_kw={'label': 'Gradient Boosting'});

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey='row');
plot_ale(proba_exp_lr, features=['shot_dist'], targets=[1], ax=ax, line_kw={'label': 'LR'});
plot_ale(proba_exp_rf, features=['shot_dist'], targets=[1], ax=ax, line_kw={'label': 'RF'});
plot_ale(proba_exp_gb, features=['shot_dist'], targets=[1], ax=ax, line_kw={'label': 'GB'});

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), sharey='row');
plot_ale(proba_exp_lr, features=['shot_dist'], targets=[0], ax=ax, line_kw={'label': 'LR'});
plot_ale(proba_exp_rf, features=['shot_dist'], targets=[0], ax=ax, line_kw={'label': 'RF'});
plot_ale(proba_exp_gb, features=['shot_dist'], targets=[0], ax=ax, line_kw={'label': 'GB'});

In [ ]:
pip install alibi-detect

In [ ]:
from alibi_detect.od import IForest
from alibi_detect.datasets import fetch_kdd
from alibi_detect.utils.data import create_outlier_batch
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.utils.visualize import plot_instance_score, plot_roc

Assume that a model is trained on normal instances of the dataset (not outliers) and standardization is applied:

In [ ]:
np.random.seed(0)
normal_batch = create_outlier_batch(X, y, n_samples=100000, perc_outlier=0)
X_train, y_train = normal_batch.data.astype('float'), normal_batch.target
print(X_train.shape, y_train.shape)
print('{}% outliers'.format(100 * y_train.mean()))

In [ ]:
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)

Apply standardization:

In [ ]:
X_train = (X_train - mean) / stdev

### Load Outlier Detector (Isolation Forest)

In [ ]:
od = IForest(threshold=None, n_estimators=100)

# train
od.fit(X_train)

Let’s assume we have some data which we know contains around 20% outliers

In [ ]:
np.random.seed(0)
perc_outlier = 20
threshold_batch = create_outlier_batch(X, y, n_samples=1000, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

In [ ]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))

### Detect Outliers

We now generate a batch of data with 10% outliers and detect the outliers in the batch.

In [ ]:
np.random.seed(1)
outlier_batch = create_outlier_batch(X, y, n_samples=1000, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target
X_outlier = (X_outlier - mean) / stdev
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

### Predict Outliers 

In [ ]:
od_preds = od.predict(X_outlier, return_instance_score=True)

### Display Results

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {:.4f}'.format(f1))

cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

In [ ]:
roc_data = {'IF': {'scores': od_preds['data']['instance_score'], 'labels': y_outlier}}
plot_roc(roc_data)

## Variable Auto-Encoders

Assume that a model is trained on normal instances of the dataset (not outliers) and standardization is applied:

In [ ]:
np.random.seed(0)
normal_batch = create_outlier_batch(X, y, n_samples=100000, perc_outlier=0)
X_train, y_train = normal_batch.data.astype('float'), normal_batch.target
print(X_train.shape, y_train.shape)
print('{}% outliers'.format(100 * y_train.mean()))

Apply standardization:

In [ ]:
mean, stdev = X_train.mean(axis=0), X_train.std(axis=0)

In [ ]:
X_train = (X_train - mean) / stdev

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Dense, InputLayer
from alibi_detect.models.losses import elbo
from alibi_detect.od import OutlierVAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_tabular, plot_roc

In [ ]:
# define model, initialize and train  outlier detector

n_features = X_train.shape[1]
latent_dim = 2

encoder_net = tf.keras.Sequential(
      [
          InputLayer(input_shape=(n_features,)),
          Dense(20, activation=tf.nn.relu),
          Dense(15, activation=tf.nn.relu),
          Dense(7, activation=tf.nn.relu)
      ])

decoder_net = tf.keras.Sequential(
      [
          InputLayer(input_shape=(latent_dim,)),
          Dense(7, activation=tf.nn.relu),
          Dense(15, activation=tf.nn.relu),
          Dense(20, activation=tf.nn.relu),
          Dense(n_features, activation=None)
      ])
# initialize outlier detector
od = OutlierVAE(threshold=None,  # threshold for outlier score
                score_type='mse',  # use MSE of reconstruction error for outlier detection
                encoder_net=encoder_net,  # can also pass VAE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                latent_dim=latent_dim,
                samples=5)
    # train
od.fit(X_train,
       loss_fn=elbo,
       cov_elbo=dict(sim=.01),
       epochs=20,
       verbose=True)

In [ ]:
np.random.seed(0)
perc_outlier = 10
threshold_batch = create_outlier_batch(X, y, n_samples=1000, perc_outlier=perc_outlier)
X_threshold, y_threshold = threshold_batch.data.astype('float'), threshold_batch.target
X_threshold = (X_threshold - mean) / stdev
print('{}% outliers'.format(100 * y_threshold.mean()))

In [ ]:
od.infer_threshold(X_threshold, threshold_perc=100-perc_outlier)
print('New threshold: {}'.format(od.threshold))

### Detect Outliers 

We now generate a batch of data with 10% outliers and detect the outliers in the batch.

In [ ]:
np.random.seed(1)
outlier_batch = create_outlier_batch(X, y, n_samples=1000, perc_outlier=10)
X_outlier, y_outlier = outlier_batch.data.astype('float'), outlier_batch.target
X_outlier = (X_outlier - mean) / stdev
print(X_outlier.shape, y_outlier.shape)
print('{}% outliers'.format(100 * y_outlier.mean()))

### Predict Outliers 

In [ ]:
od_preds = od.predict(X_outlier,
                      outlier_type='instance',    # use 'feature' or 'instance' level
                      return_feature_score=True,  # scores used to determine outliers
                      return_instance_score=True)
print(list(od_preds['data'].keys()))

### Display Results 

In [ ]:
labels = outlier_batch.target_names
y_pred = od_preds['data']['is_outlier']
f1 = f1_score(y_outlier, y_pred)
print('F1 score: {:.4f}'.format(f1))
cm = confusion_matrix(y_outlier, y_pred)
df_cm = pd.DataFrame(cm, index=labels, columns=labels)
sns.heatmap(df_cm, annot=True, cbar=True, linewidths=.5)
plt.show()

In [ ]:
plot_instance_score(od_preds, y_outlier, labels, od.threshold)

In [ ]:
roc_data = {'VAE': {'scores': od_preds['data']['instance_score'], 'labels': y_outlier}}
plot_roc(roc_data)

In [ ]:
X_recon = od.vae(X_outlier).numpy()  # reconstructed instances by the VAE

### Investigate instance-level Outlier

We can now take a closer look at some of the individual predictions on X_outlier

In [ ]:
plot_feature_outlier_tabular(od_preds,
                             X_outlier,
                             X_recon=X_recon,
                             threshold=od.threshold,
                             instance_ids=None,  # pass a list with indices of instances to display
                             max_instances=5,  # max nb of instances to display
                             top_n=5,  # only show top_n features ordered by outlier score
                             outliers_only=False,  # only show outlier predictions
                             feature_names=X.columns,  # add feature names
                             figsize=(20, 30))